<a href="https://colab.research.google.com/github/melancholymans/Bonanza/blob/master/Madelon_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このプログを読んで「ランダムフォレストと検定を用いた特徴量選択手法 Boruta」を試してみる  
https://aotamasaki.hatenablog.com/entry/2019/01/05/195813  
データはここからもらった  
http://archive.ics.uci.edu/ml/datasets/madelon  

madelon_param データファイルの概要  
madelon_train.data 2000行×特徴量500のデータファイル  
madelon_test.data 1800行×特徴量500のデータファイル  
madelon_train.labels 2000行×ラベル1のデータファイル  1か-1で構成  
madelon_valid.data 600行×特徴量500のデータファイル   
madelon_valid.label 600行×ラベル1のデータファイル  1か-1で構成  


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
base_path = 'drive/My Drive/Colab Notebooks/kaggle/Madelon_Dataset/input/'

In [35]:
!pip install boruta

     |████████████████████████████████| 61kB 2.3MB/s 


In [23]:
import numpy as np
import pandas as pd

X = pd.read_csv(base_path+'madelon_train.data',header=None,sep=' ')
X = X.iloc[:,0:-1] # 最終列が欠損値なのでカット
print(X.head())
print(X.shape)
print(X.describe())
print('-'*50)
y = pd.read_csv(base_path+'madelon_train.labels',header=None,sep=' ')
print(y.head())
print(y.shape)
print(y.describe())

   0    1    2    3    4    5    6    ...  493  494  495  496  497  498  499
0  485  477  537  479  452  471  491  ...  485  511  485  481  479  475  496
1  483  458  460  487  587  475  526  ...  338  513  486  483  492  510  517
2  487  542  499  468  448  471  442  ...  650  506  501  480  489  499  498
3  480  491  510  485  495  472  417  ...  572  454  469  475  482  494  461
4  484  502  528  489  466  481  402  ...  435  486  508  481  504  495  511

[5 rows x 500 columns]
(2000, 500)
               0            1    ...          498          499
count  2000.000000  2000.000000  ...  2000.000000  2000.000000
mean    481.722500   483.452500  ...   507.977000   490.266000
std       6.421769    30.186294  ...    37.224297    25.825273
min     462.000000   381.000000  ...   363.000000   403.000000
25%     477.000000   464.000000  ...   482.000000   473.000000
50%     482.000000   483.000000  ...   508.000000   490.000000
75%     486.000000   503.000000  ...   533.000000   507.25000

In [0]:
# train labelsデータを訓練用と検証用に分ける
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from multiprocessing import cpu_count
from boruta import BorutaPy

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.25)

In [31]:
rf = RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    n_jobs=int(cpu_count()/2)
)
rf.fit(X_train.values,y_train.values)
print(rf.classes_)
print(confusion_matrix(y_test.values,rf.predict(X_test.values),labels=rf.classes_))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[-1  1]
[[167  82]
 [ 71 180]]


# Borutaの判別

In [37]:
rf = RandomForestClassifier(n_jobs=int(cpu_count()/2),max_depth=7)
feat_selector = BorutaPy(rf,n_estimators='auto',two_step=False,verbose=2,random_state=42)
feat_selector.fit(X_train.values,y_train.values)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/boruta/boruta_py.py:384: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(X, y)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0


/usr/local/lib/python3.6/dist-packages/boruta/boruta_py.py:384: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(X, y)


Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	500
Rejected: 	0


/usr/local/lib/python3.6/dist-packages/boruta/boruta_py.py:384: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(X, y)


...

/usr/local/lib/python3.6/dist-packages/boruta/boruta_py.py:384: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(X, y)


Iteration: 	98 / 100
Confirmed: 	19
Tentative: 	1
Rejected: 	480


/usr/local/lib/python3.6/dist-packages/boruta/boruta_py.py:384: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(X, y)


Iteration: 	99 / 100
Confirmed: 	19
Tentative: 	1
Rejected: 	480


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	19
Tentative: 	1
Rejected: 	480


BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                          criterion='gini', max_depth=7,
                                          max_features='auto',
                                          max_leaf_nodes=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=90, n_jobs=1,
                                          oob_score=False,
                                          random_state=<mtrand.RandomState object at 0x7fe5fa67a5a0>,
                                          verbose=0, warm_start=False),
         max_iter=100, n_estimators='auto', perc=100,
         rando

In [0]:
Confirmed : 確認済みとの意味で　有効な特徴量を19検出した  
Tentative : 暫定が１  
Rejected : 排除が480  
  

In [0]:
X_train_selected = X_train.iloc[:,feat_selector.support_]
X_test_selected = X_test.iloc[:,feat_selector.support_]

In [39]:
rf=RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    n_jobs=int(cpu_count()/2)
)
rf.fit(X_train_selected.values, y_train.values)

print(rf.classes_)
print(confusion_matrix(y_test.values,rf.predict(X_test_selected.values),labels=rf.classes_))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[-1  1]
[[218  31]
 [ 30 221]]


前の学習の結果は153件の判定誤り  
今回は61件の判定誤りと減っている  